<a href="https://colab.research.google.com/github/VaishnaviSadija-1/CuisineConnect/blob/main/marathi_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Nov  6 10:05:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# Check GPU and install dependencies
!nvidia-smi

# Install required packages
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0
!pip install matplotlib numpy==1.24.3 scipy tensorboard Unidecode pyworld websockets soundfile
!pip install phonemizer librosa json

# Clone VITS repository
!git clone https://github.com/jaywalnut310/vits.git
%cd vits

# Install monotonic align
!pip install monotonic-align
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

Wed Nov  6 10:15:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
import json
import os
import shutil
import random

# Mount Google Drive
drive.mount('/content/drive')

# Create project directories
!mkdir -p "/content/vits/dataset/marathi"
!mkdir -p "/content/vits/filelists"

# Process the Marathi dataset
def process_marathi_data():
    # Source directory (your Google Drive folder)
    source_dir = "/content/drive/MyDrive/marathi_data"

    # Create lists to store file pairs
    data_pairs = []

    # Read all json files and their corresponding wav files
    for i in range(1, 11):  # For files 1 through 10
        json_file = f"{source_dir}/{i}.json"
        wav_file = f"{source_dir}/{i}.wav"

        if os.path.exists(json_file) and os.path.exists(wav_file):
            with open(json_file, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
                # Extract text from JSON (adjust this based on your JSON structure)
                text = json_data.get('text', '')  # Adjust this key based on your JSON structure

            data_pairs.append((wav_file, text))

    # Shuffle and split the data (80% train, 10% val, 10% test)
    random.shuffle(data_pairs)
    total = len(data_pairs)
    train_end = int(0.8 * total)
    val_end = int(0.9 * total)

    train_pairs = data_pairs[:train_end]
    val_pairs = data_pairs[train_end:val_end]
    test_pairs = data_pairs[val_end:]

    # Write filelists
    def write_filelist(pairs, filename):
        with open(f'/content/vits/filelists/{filename}', 'w', encoding='utf-8') as f:
            for wav_path, text in pairs:
                f.write(f"{wav_path}|{text}\n")

    write_filelist(train_pairs, 'marathi_train.txt')
    write_filelist(val_pairs, 'marathi_val.txt')
    write_filelist(test_pairs, 'marathi_test.txt')

    print(f"Created filelists with {len(train_pairs)} training, {len(val_pairs)} validation, and {len(test_pairs)} test samples")

# Run the processing
process_marathi_data()

Mounted at /content/drive
Created filelists with 8 training, 1 validation, and 1 test samples


In [8]:
# Clone the VITS repository
!git clone https://github.com/jaywalnut310/vits.git
%cd vits

# Install monotonic align
!pip install monotonic-align
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..


Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (81/81), 3.33 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/vits/vits/vits
/content/vits/vits/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/vits/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to al

In [9]:
# Run data processing function to prepare filelists
process_marathi_data()


Filelists created with 8 training, 1 validation, and 1 test samples


In [10]:
%cd /content/vits


/content/vits


In [11]:
!cp configs/ljs_base.json configs/marathi_config.json


In [14]:
{
  "train_files": "/content/vits/filelists/marathi_train.txt",
  "val_files": "/content/vits/filelists/marathi_val.txt",
  "sampling_rate": 22050,
  "text_cleaners": ["phonemize_cleaners"],
  "num_mels": 80,
  "n_fft": 1024,
  "hop_length": 256,
  "win_length": 1024,
  "fmin": 0,
  "fmax": 8000,
  "model": {
    "hidden_channels": 192,
    "filter_channels": 768,
    "resblock": "1",
    "resblock_dilation_sizes": [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
    "upsample_rates": [8, 8, 2, 2],
    "upsample_initial_channel": 512
  },
  "training": {
    "batch_size": 16,
    "learning_rate": 0.0002,
    "epochs": 1000,
    "validation_interval": 100,
    "checkpoint_interval": 100,
    "log_interval": 10
  }
}


{'train_files': '/content/vits/filelists/marathi_train.txt',
 'val_files': '/content/vits/filelists/marathi_val.txt',
 'sampling_rate': 22050,
 'text_cleaners': ['phonemize_cleaners'],
 'num_mels': 80,
 'n_fft': 1024,
 'hop_length': 256,
 'win_length': 1024,
 'fmin': 0,
 'fmax': 8000,
 'model': {'hidden_channels': 192,
  'filter_channels': 768,
  'resblock': '1',
  'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
  'upsample_rates': [8, 8, 2, 2],
  'upsample_initial_channel': 512},
 'training': {'batch_size': 16,
  'learning_rate': 0.0002,
  'epochs': 1000,
  'validation_interval': 100,
  'checkpoint_interval': 100,
  'log_interval': 10}}

In [15]:
!python train.py -c configs/marathi_config.json -m marathi_model


Streaming output truncated to the last 5000 lines.
DEBUG:numba.core.ssa:on stmt: return $54return_value.1
DEBUG:numba.core.ssa:==== SSA block analysis pass on 56
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x7e5881c1cfd0>
DEBUG:numba.core.ssa:on stmt: $58get_iter.1 = getiter(value=it)
DEBUG:numba.core.ssa:on stmt: $phi60.0 = $58get_iter.1
DEBUG:numba.core.ssa:on stmt: jump 60
DEBUG:numba.core.ssa:==== SSA block analysis pass on 60
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x7e5881c1cfd0>
DEBUG:numba.core.ssa:on stmt: $60for_iter.1 = iternext(value=$phi60.0)
DEBUG:numba.core.ssa:on stmt: $60for_iter.2 = pair_first(value=$60for_iter.1)
DEBUG:numba.core.ssa:on stmt: $60for_iter.3 = pair_second(value=$60for_iter.1)
DEBUG:numba.core.ssa:on stmt: $phi62.1 = $60for_iter.2
DEBUG:numba.core.ssa:on stmt: branch $60for_iter.3, 62, 104
DEBUG:numba.core.ssa:==== SSA block analysis pass on 62
DEBUG:numba.core.ssa:Running <numba.core.ssa._

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader

# Custom dataset class to load Marathi data
class MarathiDataset(Dataset):
    def __init__(self, filelist_path, transform=None):
        self.data = []
        self.transform = transform

        # Read file list (train, val, or test)
        with open(filelist_path, 'r', encoding='utf-8') as f:
            for line in f:
                wav_path, text = line.strip().split('|')
                self.data.append((wav_path, text))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        wav_path, text = self.data[idx]

        # Load the WAV file and preprocess as necessary
        audio = load_audio(wav_path)  # Define load_audio for audio processing
        # Convert text to tensor or any other desired form

        return audio, text  # Adjust based on model requirements

# Define file path for your train filelist
train_filelist_path = '/content/vits/filelists/marathi_train.txt'

# Instantiate Dataset and DataLoader
train_dataset = MarathiDataset(train_filelist_path)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)  # Adjust batch size and workers

# Now, `train_loader` can be used in the training loop



In [25]:
!pip install librosa


In [26]:
import librosa
import torch

# Define load_audio function
def load_audio(wav_path, sr=22050):
    # Load the audio file
    audio, _ = librosa.load(wav_path, sr=sr)  # You can adjust the sample rate if needed
    # Convert audio to a tensor
    audio_tensor = torch.tensor(audio)
    return audio_tensor

# Now you can use load_audio in the MarathiDataset class


In [27]:
class MarathiDataset(Dataset):
    def __init__(self, filelist_path, transform=None):
        self.data = []
        self.transform = transform

        # Read file list (train, val, or test)
        with open(filelist_path, 'r', encoding='utf-8') as f:
            for line in f:
                wav_path, text = line.strip().split('|')
                self.data.append((wav_path, text))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        wav_path, text = self.data[idx]

        # Load the WAV file
        audio = load_audio(wav_path)  # Uses the defined load_audio function
        # Process text as needed (tokenization or encoding)

        return audio, text  # Adjust as needed for your model


In [28]:
!python train.py -c configs/marathi_config.json -m marathi_model


2024-11-06 10:46:07.433249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 10:46:07.453615: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 10:46:07.459730: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 10:46:08.640164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	LOAD_FAST(arg=0, lineno=1042)
           4	LOAD_CONST(arg=1, lineno=1042)
           6	BINARY_SUBSCR(arg=None, lineno=1042)
           8	LOAD_FAST(

In [36]:
!git clone https://github.com/jaywalnut310/vits.git


fatal: destination path 'vits' already exists and is not an empty directory.


In [37]:
%cd vits


/content/vits/vits


In [39]:
!ls


attentions.py  filelists	mel_processing.py  preprocess.py     text	    utils.py
commons.py     inference.ipynb	models.py	   README.md	     train_ms.py    vits
configs        LICENSE		modules.py	   requirements.txt  train.py
data_utils.py  losses.py	monotonic_align    resources	     transforms.py


In [41]:
!pip install monotonic-align
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..


/content/vits/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_c' as 'noexcept' if you control the definition and you're sure you don't wan

In [42]:
import sys
sys.path.append('/content/vits/monotonic_align')


In [43]:
!pip install -r requirements.txt
!pip install librosa
!pip install monotonic-align


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [46]:
!apt-get install -y build-essential python3-dev



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [47]:
!pip install -r requirements.txt
!pip install librosa
!pip install monotonic-align


  Using cached Cython-0.29.21-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached librosa-0.8.0.tar.gz (183 kB)
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.3.1.tar.gz (38.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.18.5.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
!pip install numpy==1.24.3


In [2]:
!pip install librosa==0.8.0


  Using cached librosa-0.8.0.tar.gz (183 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.6 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201362 sha256=b1ba581e54ba0fd81e4974a4fce2a203367f9c7a39de44237c33fbd45843903e
  Stored in directory: /root/.cache/pip/wheels/bf/b7/85/2f8044306ccec014930aea23ad4852fca9e2584e21c6972bc6
Successfully built librosa
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.10.2.post1:
      Successfully uninstalled librosa-0.10.2.post1


In [4]:
!gdown --id <file_id>  # Replace <file_id> with the actual file ID from the Drive URL


/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `gdown --id <file_id>  # Replace <file_id> with the actual file ID from the Drive URL'


In [5]:
import os
print(os.listdir('/content/drive/MyDrive/marathi_data/'))


['1.json', '1.wav', '2.json', '2.wav', '3.json', '3.wav', '4.json', '4.wav', '5.json', '5.wav', '6.json', '6.wav', '7.json', '7.wav', '8.json', '8.wav', '9.json', '9.wav', '10.json', '10.wav']


In [7]:
import json



In [8]:
import os
import json
import random

def process_marathi_data():
    source_dir = "/content/drive/MyDrive/marathi_data"  # Your path to data
    data_pairs = []

    # For files 1 through 10
    for i in range(1, 11):
        json_file = f"{source_dir}/{i}.json"
        wav_file = f"{source_dir}/{i}.wav"

        if os.path.exists(json_file) and os.path.exists(wav_file):
            with open(json_file, 'r', encoding='utf-8') as f:
                json_data = json.load(f)  # Now it will work correctly
                text = json_data.get('text', '')  # Adjust the key based on your JSON structure

            data_pairs.append((wav_file, text))

    random.shuffle(data_pairs)
    total = len(data_pairs)
    train_end = int(0.8 * total)
    val_end = int(0.9 * total)

    # Split data into train, validation, test
    train_pairs = data_pairs[:train_end]
    val_pairs = data_pairs[train_end:val_end]
    test_pairs = data_pairs[val_end:]

    # Write filelists to disk
    def write_filelist(pairs, filename):
        with open(f'/content/vits/filelists/{filename}', 'w', encoding='utf-8') as f:
            for wav_path, text in pairs:
                f.write(f"{wav_path}|{text}\n")

    write_filelist(train_pairs, 'marathi_train.txt')
    write_filelist(val_pairs, 'marathi_val.txt')
    write_filelist(test_pairs, 'marathi_test.txt')

    print(f"Created filelists with {len(train_pairs)} training, {len(val_pairs)} validation, and {len(test_pairs)} test samples")

# Run the function to process your Marathi dataset
process_marathi_data()


Created filelists with 8 training, 1 validation, and 1 test samples


In [9]:
def process_marathi_data():
    source_dir = "/content/drive/MyDrive/marathi_data"  # Your path to data
    data_pairs = []

    # For files 1 through 10
    for i in range(1, 11):
        json_file = f"{source_dir}/{i}.json"
        wav_file = f"{source_dir}/{i}.wav"

        if os.path.exists(json_file) and os.path.exists(wav_file):
            with open(json_file, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
                text = json_data.get('text', '')  # Adjust the key based on JSON format

            data_pairs.append((wav_file, text))

    random.shuffle(data_pairs)
    total = len(data_pairs)
    train_end = int(0.8 * total)
    val_end = int(0.9 * total)

    # Split data into train, validation, test
    train_pairs = data_pairs[:train_end]
    val_pairs = data_pairs[train_end:val_end]
    test_pairs = data_pairs[val_end:]

    # Write filelists to disk
    def write_filelist(pairs, filename):
        with open(f'/content/vits/filelists/{filename}', 'w', encoding='utf-8') as f:
            for wav_path, text in pairs:
                f.write(f"{wav_path}|{text}\n")

    write_filelist(train_pairs, 'marathi_train.txt')
    write_filelist(val_pairs, 'marathi_val.txt')
    write_filelist(test_pairs, 'marathi_test.txt')

    print(f"Created filelists with {len(train_pairs)} training, {len(val_pairs)} validation, and {len(test_pairs)} test samples")

# Run the function to process your Marathi dataset
process_marathi_data()


Created filelists with 8 training, 1 validation, and 1 test samples


In [10]:
{
  "train_files": "/content/vits/filelists/marathi_train.txt",
  "val_files": "/content/vits/filelists/marathi_val.txt",
  "sampling_rate": 22050,
  "text_cleaners": ["phonemize_cleaners"],
  "num_mels": 80,
  "n_fft": 1024,
  "hop_length": 256,
  "win_length": 1024,
  "fmin": 0,
  "fmax": 8000,
  "model": {
    "hidden_channels": 192,
    "filter_channels": 768,
    "resblock": "1",
    "resblock_dilation_sizes": [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
    "upsample_rates": [8, 8, 2, 2],
    "upsample_initial_channel": 512
  },
  "training": {
    "batch_size": 16,
    "learning_rate": 0.0002,
    "epochs": 1000,
    "validation_interval": 100,
    "checkpoint_interval": 100,
    "log_interval": 10
  }
}


{'train_files': '/content/vits/filelists/marathi_train.txt',
 'val_files': '/content/vits/filelists/marathi_val.txt',
 'sampling_rate': 22050,
 'text_cleaners': ['phonemize_cleaners'],
 'num_mels': 80,
 'n_fft': 1024,
 'hop_length': 256,
 'win_length': 1024,
 'fmin': 0,
 'fmax': 8000,
 'model': {'hidden_channels': 192,
  'filter_channels': 768,
  'resblock': '1',
  'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
  'upsample_rates': [8, 8, 2, 2],
  'upsample_initial_channel': 512},
 'training': {'batch_size': 16,
  'learning_rate': 0.0002,
  'epochs': 1000,
  'validation_interval': 100,
  'checkpoint_interval': 100,
  'log_interval': 10}}

In [12]:
!python train.py -c configs/marathi_config.json -m marathi_model


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [13]:
%cd vits


/content/vits


In [14]:
!ls


attentions.py  filelists	mel_processing.py  __pycache__	     train_ms.py
commons.py     inference.ipynb	models.py	   README.md	     train.py
configs        LICENSE		modules.py	   requirements.txt  transforms.py
dataset        logs		monotonic_align    resources	     utils.py
data_utils.py  losses.py	preprocess.py	   text		     vits


In [15]:
!git clone https://github.com/jaywalnut310/vits.git
%cd vits


fatal: destination path 'vits' already exists and is not an empty directory.
/content/vits/vits


In [16]:
!python3 train.py -c configs/marathi_config.json -m marathi_model


2024-11-06 11:10:44.391071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:10:44.412019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:10:44.418130: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:10:44.433733: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:10:45.607136: W tensorflow/comp

In [17]:
dtype=np.complex  # Old line


NameError: name 'np' is not defined

In [18]:
import numpy as np


In [19]:
dtype=complex  # or dtype=np.complex128


In [20]:
import numpy as np

# Old code that uses np.complex
dtype=np.complex


AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [21]:
dtype = complex


In [22]:
dtype = np.complex128


In [23]:
import numpy as np
dtype = np.complex  # This will raise an error in NumPy 1.20+


AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [24]:
dtype = complex  # Use the built-in complex type


In [25]:
dtype = np.complex128  # Use NumPy's complex128


In [27]:
import numpy as np
dtype = np.complex128  # Correct replacement for np.complex


In [28]:
dtype = complex  # Python's built-in complex type


In [29]:
!python3 train.py -c configs/marathi_config.json -m marathi_model


2024-11-06 11:14:55.279939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:14:55.300226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:14:55.306320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:14:55.321294: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:14:56.485096: W tensorflow/comp